In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Total UK Cardholder Dometic Online Monthly

# Summarise the data by UK Cardholder Online Spending Total Monthly Domestics

UK_spending_Online_All = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Month' 
and mcg = 'All'
and merchant_channel = 'Online' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'UNITED KINGDOM' 
GROUP BY destination_country, 
time_period_value 
ORDER BY time_period_value'''
df_by_Online_All = bq.read_bq_table_sql(client, UK_spending_Online_All)
df_by_Online_All = df_by_Online_All.rename(columns={'total_spend': 'Online_spend_All'})
df_by_Online_All.head()

print(df_by_Online_All)

# Save the DataFrame to a CSV file
csv_filename = "UK_Online_Spending_Mth_Dom.csv"
df_by_Online_All.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")

In [ ]:
# Total UK Cardholder International Online Monthly

# Summarise the data by UK Cardholder Online Spending Total Monthly Abroad

UK_spending_Online_All = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Month' 
and mcg = 'All'
and merchant_channel = 'Online' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM' 
GROUP BY destination_country, 
time_period_value 
ORDER BY time_period_value'''
df_by_Online_All = bq.read_bq_table_sql(client, UK_spending_Online_All)
df_by_Online_All = df_by_Online_All.rename(columns={'total_spend': 'Online_spend_All'})
df_by_Online_All.head()

print(df_by_Online_All)

# Save the DataFrame to a CSV file
csv_filename = "UK_Online_Spending_Mth_Intl.csv"
df_by_Online_All.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")

In [ ]:
# Summarise the data by UK Cardholder Spending All Monthly --------------- Cardholders' Number Total Monthly ----  mcg = 'All'

UK_spending_by_mcg_All = '''SELECT time_period_value, cardholders, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Month' 
and mcg = 'All' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
 
GROUP BY cardholders, destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_mcg_All = bq.read_bq_table_sql(client, UK_spending_by_mcg_All)
df_by_mcg_All.head()

# Caculate UK Domestic Total Spending Quarterly

# Assuming df_by_mcg_All is the DataFrame returned from the BigQuery query
# Then group by 'time_period_value' and sum the 'spend' for each quarter

# Check if df_by_mcg_All is not None and has the expected columns
if df_by_mcg_All is not None and 'time_period_value' in df_by_mcg_All.columns and 'spend' in df_by_mcg_All.columns:
    # Group by quarter and sum the spend
    UK_spending_by_mcg_All = df_by_mcg_All.groupby('time_period_value')['cardholders'].sum().reset_index()
   
 # Rename the column
    UK_spending_by_mcg_All = UK_spending_by_mcg_All.rename(columns={'cardholders': 'Spend_mcg_All_cardholders'})
    print(UK_spending_by_mcg_All)
else:
    print("DataFrame is empty or missing required columns.")
    
    
# Save to CSV
UK_spending_by_mcg_All.to_csv("UK_Spend_Mth_All_cardholders.csv", index=False)
